In [1]:
import json
import folium
from datetime import datetime

# Путь к файлу с камерами
file_path = '/content/drive/MyDrive/useful/intersections/Actual/intersecting_groups.json'

try:
    # Чтение файла
    with open(file_path, 'r', encoding='utf-8') as file:
        cameras = json.load(file)
except FileNotFoundError:
    print(f"Файл {file_path} не найден.")
    exit(1)
except json.JSONDecodeError:
    print(f"Ошибка при чтении JSON из файла {file_path}.")
    exit(1)

# Подсчёт количества камер
camera_count = len(cameras)

# Текущая дата или можно указать конкретную дату составления
creation_date = datetime.now().strftime("%d.%m.%Y")  # Например: 05.04.2025
# Если хотите зафиксировать дату, замените на: "01.01.2025"

# Создание карты
m = folium.Map(location=[59.835691, 30.384718], zoom_start=15)  # Центральная точка и начальный зум

# Словарь цветов для первых цифр типов камер
camera_colors = {
    '1': 'purple',
    '2': 'darkviolet',
    '3': 'indigo',
    '4': 'purple',
    '5': 'orange',
    '6': 'black',
    '7': 'brown',
    '8': 'gray'
}

# Добавление полигонов и кругов для каждой камеры
for camera in cameras:
    required_keys = ['lat', 'lng', 'name', 'district', 'address', 'cameraType', 'defaultAzimuth', 'lat_1', 'lng_1', 'lat_2', 'lng_2']
    if not all(key in camera for key in required_keys):
        print(f"Пропущены необходимые ключи для камеры {camera.get('name', 'без имени')}")
        continue

    # Определяем первую цифру типа камеры
    first_digit = camera['cameraType'][0] if camera['cameraType'] else ''

    # Определяем цвет для первой цифры типа камеры
    color = camera_colors.get(first_digit, 'gray')

    # Добавление треугольника для зоны обзора с попапом
    folium.Polygon(
        locations=[
            [camera['lat'], camera['lng']],
            [camera['lat_1'], camera['lng_1']],
            [camera['lat_2'], camera['lng_2']],
            [camera['lat'], camera['lng']]
        ],
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.3,
        popup=(
            f"Имя камеры: {camera['name']}<br>"
            f"Район: {camera['district']}<br>"
            f"Адрес: {camera['address']}<br>"
            f"Тип камеры: {camera['cameraType']}<br>"
            f"Азимут: {camera['defaultAzimuth']}°"
        )
    ).add_to(m)

    # Добавление круга диаметром 1 метр в вершине камеры с попапом
    folium.Circle(
        location=[camera['lat'], camera['lng']],
        radius=1,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=(
            f"Камера {camera['name']}<br>"
            f"Тип: {camera['cameraType']}<br>"
            f"Район: {camera['district']}<br>"
            f"Адрес: {camera['address']}<br>"
            f"Азимут: {camera['defaultAzimuth']}°"
        )
    ).add_to(m)


# Сохранение карты в HTML файл
map_file_path = '/content/drive/MyDrive/useful/intersections/Actual/all_cameras_map.html'
m.save(map_file_path)

print(f"Карта для всех камер сохранена в {map_file_path}")

# Перезапись карты добавляя футер
# Путь к уже сохранённому HTML-файлу
map_file_path = '/content/drive/MyDrive/useful/intersections/Actual/all_cameras_map.html'

# Чтение файла
with open(map_file_path, 'r', encoding='utf-8') as f:
    html_content = f.read()

# Проверим, не добавлен ли уже футер (защита от дублирования)
if "</body>" in html_content:
    # Вставляем перед </body>
    footer_html = f"""
    <div style="position: fixed; bottom: 0; left: 0; width: 100%; background-color: white; color: black;
                text-align: center; padding: 6px; font-family: Arial, sans-serif; font-size: 13px;
                font-weight: bold; z-index: 9999; border-top: 2px solid #aaa; box-shadow: 0 -2px 5px rgba(0,0,0,0.1);
                pointer-events: none; display: flex; justify-content: center; align-items: center; height: 30px;">
        <div style="pointer-events: auto;">
            Всего камер: {camera_count} | Дата составления файла: {creation_date}
        </div>
    </div>
    """
    html_content = html_content.replace("</body>", footer_html + "\n</body>")

    # Перезаписываем файл
    with open(map_file_path, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"Футер добавлен, файл перезаписан: {map_file_path}")
else:
    print("Ошибка: не удалось найти место для вставки футера в HTML.")

Карта для всех камер сохранена в /content/drive/MyDrive/useful/intersections/Actual/all_cameras_map.html
Футер добавлен, файл перезаписан: /content/drive/MyDrive/useful/intersections/Actual/all_cameras_map.html
